# Etapas

1. Mudar o tipo das variáveis da **base original**

2. Criar nova base com mudança de granularidade

3. Para cada Tipo de Tratamento de Missing Values - Mínimo, Máximo e a Média 

   1. Tratar *Missing Values* da base de nova granularidade

   2. Tratar *Missing values* na **base original**

   3. Criar variáveis na **base de nova granulariadade** com **relação a base original**

   4. Variáveis Dummies, Normalização e Seleção de Variáveis

   5. Salvar as novas bases

# Sobre o Projeto

   [Especificação do Projeto](https://github.com/AndersonAlencarBarros/IC/blob/main/Previs%C3%A3o%20de%20Pedidos%20em%20uma%20Loja%20Online/DadosDoProjeto/EspecificacaoProjeto2.pdf)

   [DATA MINING CUP Competition 2013](https://github.com/AndersonAlencarBarros/IC/blob/main/Previs%C3%A3o%20de%20Pedidos%20em%20uma%20Loja%20Online/DadosDoProjeto/dmc2013_task.pdf)

   [Features](https://github.com/AndersonAlencarBarros/IC/blob/main/Previs%C3%A3o%20de%20Pedidos%20em%20uma%20Loja%20Online/DadosDoProjeto/features.pdf.pdf)

      



###**Importando as bibliotecas**

In [2]:
import numpy as np
import pandas as pd

###**Carregando a base de treinamento**

In [3]:
url = 'https://raw.githubusercontent.com/AndersonAlencarBarros/IC/main/Previs%C3%A3o%20de%20Pedidos%20em%20uma%20Loja%20Online/Bases/transact_train.txt'
transact_train = pd.read_csv(url, sep='|')
transact_train

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,bSumPrice,bStep,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.000,1,59.99,59.99,59.99,1,59.99,59.99,59.99,?,?,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.940,1,59.99,59.99,59.99,1,59.99,59.99,59.99,2,y,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,59.99,?,y,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.000,0,?,?,?,0,?,?,?,2,y,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15.633,0,?,?,?,0,?,?,?,?,y,completely orderable,?,?,?,?,?,?,?,?,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429008,49998,18,7,5988.882,77,9.99,149.99,5253.28,3,49.95,99.99,209.93,?,?,?,?,?,?,?,?,?,?,?,n
429009,49999,18,7,675.114,6,59.0,199.99,509.96,1,89.99,89.99,89.99,?,?,?,25038,?,?,?,0,24,1,4,n
429010,49999,18,7,715.341,7,59.0,649.99,1159.95,1,89.99,89.99,89.99,1,y,completely orderable,25038,?,?,?,0,24,1,4,n
429011,50000,18,7,0.000,1,6.99,6.99,6.99,1,6.99,6.99,6.99,?,?,?,47,300,537,5,0,53,1,34,n


### **Carregando a Base de Testes**

In [4]:
url = 'https://raw.githubusercontent.com/AndersonAlencarBarros/IC/main/Previs%C3%A3o%20de%20Pedidos%20em%20uma%20Loja%20Online/Bases/transact_class.txt'
transact_test = pd.read_csv(url, sep='|')
transact_test

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,bSumPrice,bStep,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
0,1,18,7,136.833,3,39.99,39.99,79.98,1,39.99,39.99,39.99,2,y,completely orderable,25039,1300,489,188,5,49,1,65
1,1,18,7,189.984,3,39.99,39.99,79.98,1,39.99,39.99,39.99,?,y,completely orderable,25039,1300,489,188,5,49,1,65
2,1,18,7,342.894,6,16.99,39.99,113.96,2,16.99,39.99,56.98,?,?,?,25039,1300,489,188,5,49,1,65
3,1,18,7,411.051,8,16.99,39.99,149.94,3,16.99,39.99,74.97,?,?,?,25039,1300,489,188,5,49,1,65
4,1,18,7,460.049,10,16.99,39.99,189.92,4,16.99,39.99,94.96,?,?,?,25039,1300,489,188,5,49,1,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45063,5111,5,1,26.286,3,4.99,4.99,9.98,1,4.99,4.99,4.99,3,y,completely orderable,?,?,?,?,?,?,?,?
45064,5111,5,1,30.059,3,4.99,4.99,9.98,1,4.99,4.99,4.99,4,y,completely orderable,?,?,?,?,?,?,?,?
45065,5111,5,1,33.388,3,4.99,4.99,9.98,1,4.99,4.99,4.99,5,y,completely orderable,?,?,?,?,?,?,?,?
45066,5111,5,1,51.641,3,4.99,4.99,9.98,1,4.99,4.99,4.99,4,y,completely orderable,?,?,?,?,?,?,?,?


### **Tratamento de Missing Values**

In [ ]:
class PreProcess:
    def __init__(self, data, treatment):
        self.data = data
        self.treatment = treatment
        self.numeric_columns = ([
          'cMinPrice',
          'cMaxPrice',
          'cSumPrice',
          'bMinPrice',
          'bMaxPrice',
          'bSumPrice',
          'bStep',
          'maxVal',
          'customerScore',
          'accountLifetime',
          'payments',
          'age',
          'address',
          'lastOrder' ])

        self.categorial_columns = [
          'onlineStatus',      
          'availability',              
        ]
        
    def change_dtype(self):
      to_float = ['cMinPrice', 'cMaxPrice', 'cSumPrice', 
                  'bMinPrice', 'bMaxPrice', 'bSumPrice']
                  
      to_int = ['bStep', 'maxVal', 'customerScore', 
                'accountLifetime', 'age', 'address', 'lastOrder', 'payments',
                'sessionNo', 'startHour', 'startWeekday', 'cCount', 'bCount']

      self.data[to_float] = self.data[to_float].apply(pd.to_numeric, errors='coerce', downcast='float')
      self.data[to_int] = self.data[to_int].apply(pd.to_numeric, errors='coerce', downcast='float')
        
    def replace_MissingValue(self):
      self.data.replace('?', np.nan, inplace=True)
     
    def categorical_treatment_Method(self):
      for i in self.categorial_columns:
        self.data[i].fillna("desconhecida", inplace = True)
 
    def numerical_treatment_Method(self):
      if self.treatment == 'min':
        for i in self.numeric_columns:
          self.data[i].fillna(self.data[i].min(), inplace = True)
      elif self.treatment == 'max': 
        for i in self.numeric_columns:
          self.data[i].fillna(self.data[i].max(), inplace = True) 
      elif self.treatment == 'mean':  
        for i in self.numeric_columns:
          self.data[i].fillna(self.data[i].mean(), inplace = True)
      elif self.treatment == 'drop':
        self.data = self.data.dropna()
        self.data.reset_index(drop=True, inplace=True)

    def change_granularity(self):
      data_groups = self.data.groupby("sessionNo")
      data_new_granularity = data_groups.tail(1)
      new_data = data_new_granularity.reset_index(drop=True)
      
      return new_data

    def dropColumn(self, column):
      self.data.drop(column, inplace=True, axis=1)

    def create_dummies_variables(self):
      self.data = pd.get_dummies(self.data, prefix_sep = '_ ', drop_first = True)

In [ ]:
# Parâmetros Principais

missing_value_treatment = 'min'
file_train_name = 'Train_Min.csv'
file_test_name = 'Test_Min.csv'

## **Tratamento dos Missing Value**

In [ ]:
transact_train_original = PreProcess(transact_train, missing_value_treatment)
transact_test_original = PreProcess(transact_test, missing_value_treatment)

transact_train_original.replace_MissingValue()
transact_test_original.replace_MissingValue()

transact_train_original.change_dtype()
transact_test_original.change_dtype()
 
transact_train_original.categorical_treatment_Method()
transact_test_original.categorical_treatment_Method()

# Remover linha com order igual a NaN
transact_train_original.data = transact_train_original.data.iloc[:-1]

###**Mudando a Granularidade**

A variável alvo desse problema é determinar a probabilidade de compra de um visitante, que corresponde a um registro da sessão. Porém os registros da base de dados e da base de testes está representa por transação. Assim vamos mudar a granularidade de transação para sessão. 

In [ ]:
transact_test_gran = transact_test_original.change_granularity()
transact_train_gran = transact_train_original.change_granularity()

In [ ]:
transact_test_gran_class = PreProcess(transact_test_gran, missing_value_treatment)
transact_train_gran_class = PreProcess(transact_train_gran, missing_value_treatment)

In [ ]:
# Tratamento de Missing Value na base de Nova Granularidade

transact_train_gran_class.numerical_treatment_Method()
transact_test_gran_class.numerical_treatment_Method()

###**Criação de Novas Variáveis**

- cMedSumPrice   
- bMedSumPrice
- cInterval = cMaxPrice - cMinPrice	
- bInterval = bMaxPrice - bMinPrice	
- new_user = if (payments == 0) then

In [ ]:
# Tratamento de Missing Value na base original

transact_test_original.numerical_treatment_Method()
transact_train_original.numerical_treatment_Method()

In [ ]:
transact_test_original.data.isna().sum()

sessionNo              0
startHour              0
startWeekday           0
duration               0
cCount                 0
cMinPrice              0
cMaxPrice              0
cSumPrice              0
bCount                 0
bMinPrice              0
bMaxPrice              0
bSumPrice              0
bStep                  0
onlineStatus           0
availability           0
customerNo         17264
maxVal                 0
customerScore          0
accountLifetime        0
payments               0
age                    0
address                0
lastOrder              0
dtype: int64

In [ ]:
transact_train_original.data.isna().sum()

sessionNo               0
startHour               0
startWeekday            0
duration                0
cCount                  0
cMinPrice               0
cMaxPrice               0
cSumPrice               0
bCount                  0
bMinPrice               0
bMaxPrice               0
bSumPrice               0
bStep                   0
onlineStatus            0
availability            0
customerNo         151098
maxVal                  0
customerScore           0
accountLifetime         0
payments                0
age                     0
address                 0
lastOrder               0
order                   0
dtype: int64

In [ ]:
# MedSumPrice
def create_MedSumPrice(data_g, data_or, variable_name, column):
  data_groups = data_or.groupby("sessionNo")
  
  if column == 'cSumPrice':
    variable_data = data_groups.cSumPrice.mean()
  else:
    variable_data = data_groups.bSumPrice.mean()
  
  variable_data.reset_index(inplace=True, drop=True)
  data_g[variable_name] = variable_data
  return data_g

In [ ]:
# Interval = MaxPrice - MinPrice	
def create_Interval(data_g, data_or, variable_name, columns):
  data_groups = data_or.groupby("sessionNo")
  variable_data = (data_groups[columns[0]].max() - data_groups[columns[1]].min())
  variable_data.reset_index(inplace=True, drop=True)
  data_g[variable_name] = variable_data

  return data_g

In [ ]:
# Novo Usuário - payments == 0
def create_new_user(data_g, data_or, column):
  data_groups = data_or.groupby("sessionNo")
  data_g['new_user'] = np.where(data_groups[column] == 0, 1, 0)

  return data_g

In [ ]:
## Criação de Variáveis para base Teste 

# MedSumPrice
transact_test_gran_class.data = create_MedSumPrice(transact_test_gran_class.data, transact_test_original.data, 'cMedSumPrice', 'cSumPrice')
transact_test_gran_class.data = create_MedSumPrice(transact_test_gran_class.data, transact_test_original.data, 'bMedSumPrice', 'bSumPrice')

# Interval
transact_test_gran_class.data = create_Interval(transact_test_gran_class.data, transact_test_original.data, 'bInterval', ['bMaxPrice', 'bMinPrice'])
transact_test_gran_class.data = create_Interval(transact_test_gran_class.data, transact_test_original.data, 'cInterval', ['cMaxPrice', 'cMinPrice'])

# New User
transact_test_gran_class.data = create_new_user(transact_test_gran_class.data, transact_test_original.data, 'payments')

In [ ]:
## Criação de Variáveis para base de Treinamento 

# MedSumPrice
transact_train_gran_class.data = create_MedSumPrice(transact_train_gran_class.data, transact_train_original.data, 'cMedSumPrice', 'cSumPrice')
transact_train_gran_class.data = create_MedSumPrice(transact_train_gran_class.data, transact_train_original.data, 'bMedSumPrice', 'bSumPrice')

# Interval
transact_train_gran_class.data = create_Interval(transact_train_gran_class.data, transact_train_original.data, 'cInterval', ['cMaxPrice', 'cMinPrice'])
transact_train_gran_class.data = create_Interval(transact_train_gran_class.data, transact_train_original.data, 'bInterval', ['bMaxPrice', 'bMinPrice'])

# New User
transact_train_gran_class.data = create_new_user(transact_train_gran_class.data, transact_train_original.data, 'payments')

### **Transformação**

In [ ]:
# Remover e separar 'order'
transact_train_gran_class.data['order'].replace({'y': 1, 'n': 0}, inplace=True)
order = transact_train_gran_class.data['order']
transact_train_gran_class.dropColumn('order')

In [ ]:
# Remover 'customerNo'

transact_train_gran_class.dropColumn('customerNo')
transact_test_gran_class.dropColumn('customerNo')

In [ ]:
# Variáveis dummies

transact_train_gran_class.create_dummies_variables()
transact_test_gran_class.create_dummies_variables()

In [ ]:
# Drop 'availability_ mainly not determinable'

transact_train_gran_class.dropColumn('availability_ mainly not determinable')

In [ ]:
# Normalizar  

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()

train_norm = scaler.fit_transform(transact_train_gran_class.data)
test_norm = scaler.transform(transact_test_gran_class.data)

x_train_norm_df = pd.DataFrame(train_norm, columns=transact_train_gran_class.data.columns)
x_test_norm_df = pd.DataFrame(test_norm, columns=transact_test_gran_class.data.columns)

# Atualizar com os dados normalizado
transact_train_gran_class.data = x_train_norm_df
transact_test_gran_class.data = x_test_norm_df

### **Salvar Arquivos**

In [ ]:
def download_file(data, name_file):
  from google.colab import files
  data.to_csv(name_file, index = False)
  files.download(name_file)

In [ ]:
download_file(transact_test_gran_class.data, file_test_name)
download_file(transact_train_gran_class.data, file_train_name)
# download_file(order, 'order.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>